## CH3.5 가상 문서 생성하기

In [ ]:
### Step1-1. 저장 된 벡터 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch03_vector_index_storage") 
vector_index = load_index_from_storage(storage_context)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\index_store.json.


In [ ]:
### Step1-2. LLM 설정

## API KEY 설정
from dotenv import load_dotenv
load_dotenv()

## 모델 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5,)

In [3]:
### Step2-1. 가상문서 변환 객체 생성
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
hyde = HyDEQueryTransform(include_original=True,# 원본 쿼리 문자열도 함께 임베딩에 포함할지 여부 True
                          hyde_prompt=None) # 기본 프롬프트 사용
print("### Prompt : ", hyde._hyde_prompt)
print("### Type : ", type(hyde._hyde_prompt))

### Prompt :  metadata={'prompt_type': <PromptType.SUMMARY: 'summary'>} template_vars=['context_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='Please write a passage to answer the question\nTry to include as many key details as possible.\n\n\n{context_str}\n\n\nPassage:"""\n'
### Type :  <class 'llama_index.core.prompts.base.PromptTemplate'>


In [4]:
### Step2-2. 가상문서 생성 프롬프트 작성()
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
custom_hyde_prompt = PromptTemplate(
    # template에서 취급 변수 지정
    template="당신은 금융 용어에 대한 전문가입니다. 사용자 질문에 대한 가상의 답변을 100자 이내로 요약해주세요.\n{context_str}") 
hyde = HyDEQueryTransform(include_original=True,
                          hyde_prompt=custom_hyde_prompt)


In [ ]:
### Step3-1. 사용자 쿼리를 입력받아 가상문서 생성하기
query_str = "기저효과에 대해서 알려주세요." 
transformed_query = hyde.run(query_str)

In [ ]:
# 결과 확인
print("=== 입력 Query ===")
print(transformed_query)
print("Type :", type(transformed_query))
print("\n=== 생성된 가상 문서 목록 ===")
for i, doc_str in enumerate(transformed_query.custom_embedding_strs):
    print(f"[{i}] {doc_str}")

=== 입력 Query ===
기저효과에 대해서 알려주세요.
Type : <class 'llama_index.core.schema.QueryBundle'>

=== 생성된 가상 문서 목록 ===
[0] 기저효과는 비교 대상 기간의 수치가 비정상적으로 높거나 낮아 현재 수치의 변동이 과장되어 보이는 현상입니다.
[1] 기저효과에 대해서 알려주세요.


In [8]:
### query engine 객체 생성
query_engine = vector_index.as_query_engine()

### Original Query 사용시

In [9]:
print(f'Original Query : {query_str}')
response_original = query_engine.query(query_str)
print(f"Response :\n{response_original}")

Original Query : 기저효과에 대해서 알려주세요.
Response :
기저효과는 특정 시점의 경제 지표나 통계 수치가 이전 시점의 낮거나 높은 수치에 의해 왜곡되어 나타나는 현상을 말합니다. 예를 들어, 전년도에 비해 현재의 경제 성장률이 크게 상승한 것처럼 보일 수 있지만, 이는 전년도의 수치가 매우 낮았기 때문일 수 있습니다. 따라서 기저효과는 통계 수치를 해석할 때 주의해야 할 요소 중 하나입니다.


In [10]:
from llama_index.core.query_engine import TransformQueryEngine
hyde_query_engine = TransformQueryEngine(query_engine=query_engine, 
                                         query_transform=hyde)


### Hyde 사용시

In [11]:
response = hyde_query_engine.query(query_str)
print(response)

기저효과는 경제지표의 증가율을 해석할 때 기준시점과 비교시점의 상대적인 위치에 따라 경제상황에 대한 평가가 왜곡될 수 있는 현상을 말합니다. 호황기의 경제 상황을 기준으로 현재를 비교하면 경제지표가 실제보다 위축된 모습으로 나타날 수 있으며, 반대로 불황기의 상황을 기준으로 하면 지표가 실제보다 부풀려질 수 있습니다. 이는 물가상승률을 설명할 때 자주 사용되며, 특정 시점의 특수한 상황이 비교시점의 지표에 어떤 영향을 미치는지 고려하는 것이 중요합니다.
